# Comparing a USGSCSM and ISIS camera for MRO CTX

In [1]:
import pvl
import numpy as np
import os
import pandas as pd
import knoten
import csmapi
import json

os.environ['ISISROOT'] = '/usgs/pkgs/isis3.8.0_RC1/install'
from pysis import isis
from pysis.exceptions import ProcessError

import ctypes

lib = ctypes.CDLL('/home/acpaquette/repos/usgscsm/build/libusgscsm.so')
print(lib)
if not lib:
    warnings.warn('Unable to load usgscsm shared library')

<CDLL '/home/acpaquette/repos/usgscsm/build/libusgscsm.so', handle 55b950c963d0 at 0x2b38bbeae1d0>


/home/acpaquette/repos/knoten/knoten/__init__.py:15: UserWarning: libusgscsm not installed, unable to load shared library.
  warnings.warn('libusgscsm not installed, unable to load shared library.')
/home/acpaquette/repos/knoten/knoten/__init__.py:20: UserWarning: Unable to load usgscsm shared library
  warnings.warn('Unable to load usgscsm shared library')


## Make a CSM sensor model

In [2]:
imageLink = 'https://pdsimage.wr.usgs.gov/Missions/Dawn/Ceres/DWNCSFC2_1A/DATA/IMG/20150604_SURVEY/20150617_CYCLE5/FC21A0038582_15170161546F6F.IMG'
# !wget -N -P data/ {imageLink}
fileName = 'data/' + os.path.split(imageLink)[1]
!curl {imageLink} > {fileName}
    
camera = knoten.csm.create_csm(fileName)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2150k  100 2150k    0     0  2590k      0 --:--:-- --:--:-- --:--:-- 2594k


## Ingest the image and spiceinit

In [3]:
# Set the output location of the resulting .cub
cub_loc = os.path.splitext(fileName)[0] + '.cub'

try: 
    isis.dawnfc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid')
except ProcessError as e:
    print(e.stderr)

## Define a function that compares ISIS and USGSCSM pixels

In [4]:
def check_pixel(camera, cub, line, sample):
    """Compares ISIS and USGSCSM pixel.
    
    Takes an image coordinate, projects it to a ground point using ISIS, then projects
    the result back into an image coordinate using USGSCSM and computes the difference
    between image coordinates.
    """
    output = isis.campt(from_=cub, line=line, sample=sample)
    pvl_output = pvl.loads(output)
    bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
    bodyfixed = np.asarray(bodyfixed.value) * 1000
    image_coord = camera.groundToImage(csmapi.EcefCoord(*bodyfixed))
    # (.5,.5) in CSM == (1,1) in ISIS, so we have to subtract (.5,.5) from the ISIS pixels
    line_diff = line - image_coord.line - .5
    sample_diff = sample - image_coord.samp - .5
    return line_diff, sample_diff

## Get the total number of lines / samples

In [5]:
isis_label = pvl.load(cub_loc)
n_samples = isis_label['IsisCube']['Core']['Dimensions']['Samples']
n_lines = isis_label['IsisCube']['Core']['Dimensions']['Lines']

## Compare top left, top right, bottom left, bottom right, and center pixels using check_pixel

In [6]:
pixels_dict = {'line' : [1,1,n_lines, n_lines, n_lines/2],
               'sample' : [1, n_samples, 1, n_samples, n_samples/2]}

pixels_df = pd.DataFrame.from_dict(pixels_dict)
pixels_df['line_diff'] = np.NaN
pixels_df['sample_diff'] = np.NaN

for idx, row in pixels_df.iterrows():
    pixels_df.iloc[idx]['line_diff'], pixels_df.iloc[idx]['sample_diff'] = check_pixel(camera, cub_loc, row['line'], row['sample'])

pixels_df

,line,sample,line_diff,sample_diff
0,1.0,1.0,0.727812,1.551868
1,1.0,1024.0,0.268634,1.160999
2,1024.0,1.0,0.951262,2.010392
3,1024.0,1024.0,0.497614,1.622388
4,512.0,512.0,0.570364,1.567242
